# Outlook

Having demonstrated the 1-D and 2-D case, we are ready to move on to more complicated quantum walks that actually exhibit a quantum speed up. Fortunately [1], has a circuit for implementing a transformed quantum walk problem, but the actual walking for the graph of interest needs to be implemented by a black box function. The paper doesn't illustrate how to implement the actual black box function, so we came up with an algorithmic way to implement any black box function.

Once that's done, it's straightforward to include the rest of the quantum gates necessary to implement exponentially sped up quantum walks on arbitrary quantum graphs. One more hack-a-thon should be enough to implement arbitrary quantum walks on a quantum computer!

## Implementing Black Box Problems

In the notebook, below we provide the code for defining black box functions in the context of [1], black box functions that encode graphs.

### Black Box Problems
Black box problems are problems where some function is implemented without knowing the details of how it works. The problem is then to determine some characteristic of the black box function. Maybe some particular input that creates a desired output, or determining something about the function that summarizes it, for example like in the Deutsch-Josza algorithm, or Simon's algorithm.

[6] E. Bernstein and U. Vazirani. Quantum complexity theory. SIAM J. on Computing, 26:1411–1473, 1997.

[10] D. Deutsch. Quantum theory, the Church-Turing principle, and the universal quantum computer. Proc. Roy. Soc. London A, 400:96–117, 1985.

[11] D. Deutsch and R. Jozsa. Rapid solution of problems by quantum computation. Proc. Roy. Soc. London A, 439:553–558, 1992.

[26] D. Simon. On the power of quantum computation. SIAM J. on Computing, 26:1474–1483, 1997.

### Examples of Black Box Functions
U |a, b>  = | a, b XOR f(a)>


# Citations
[1]: A. M. Childs, et al. Exponential Algorithmic Speedup by a Quantum Walk

[2]: https://arxiv.org/pdf/0706.0304.pdf

In [1]:
# Library for implementing XOR
import qiskit
import numpy as np
from qiskit import(
    IBMQ, QuantumCircuit, ClassicalRegister, QuantumRegister, execute,
    QuantumCircuit,
    execute,
    Aer)
from qiskit.visualization import plot_histogram, plot_state_city

In [2]:
if __name__ == "__main__":
    pass

In [3]:
def binary_it(alpha):
    ''' binary_it:
            returns a list that is indexed according to power,
                binaried[0] = is LSB.
    '''
    binaried = [int(x) for x in bin(alpha)[2:]]
    binaried.reverse()
    return binaried

def invert(alpha):
    return [ 1 - x for x in alpha]

# Implementation of Black Box Functions

In [4]:
def xor_gate_v_each(circuit,   a,b,r,   v,c,a0):
    ''' xor_gate_v_each:
            See paper 'An Example of the Difference Between Quantum and Classical Random Walks'
        
            args:
                circuit -- the py object that holds the quantum circuit.
                
                a -- q. reg. for the vertex.
                b -- q. reg. for holding neighbor vertex.
                r -- q. reg. for checking valid edge.
                
                f(c, a0) -- function that returns the neighboring
                         -- vertex of a0 using label c for the vertex label.
                c        -- label of the edge
                a0       -- label of the vertex
    '''
    
    #vc = v(c,a0);
    vc = 1
    
    #build in the not's for the a0 check.
    binaried = binary_it(a0)
        
    #match binaried
    if len(binaried) > len(a):
        binaried = binaried[:len(a)]
    elif len(a) > len(binaried):
        for i in range(len(a)-len(binaried)):
            binaried.append(0)
    
    notMask = invert(binaried)
    
    for i in range(len(notMask)):
        if notMask[i]:
            circuit.x(a[i]);
    
    
    #build in the cnots for xoring b
    binaried = binary_it(vc)
        
    #match binaried
    if len(binaried) > len(a):
        binaried = binaried[:len(a)]
    elif len(a) > len(binaried):
        for i in range(len(a)-len(binaried)):
            binaried.append(0)
    
    for i in range(len(binaried)):
        if binaried[i]:
            circuit.mct(a,b[i], None, mode='noancilla')
        
    #build in the not's for the a0 return.
    for i in range(len(notMask)):
        if notMask[i]:
            circuit.x(a[i]);

In [5]:
def xor_gate_f_each(circuit,   a,b,r,   v,c,a0):
    ''' xor_gate_v_each:
            See paper 'An Example of the Difference Between Quantum and Classical Random Walks'
        
            args:
                circuit -- the py object that holds the quantum circuit.
                
                a -- q. reg. for the vertex.
                b -- q. reg. for holding neighbor vertex.
                r -- q. reg. for checking valid edge.
                
                f(c, a0) -- function that returns the neighboring
                         -- vertex of a0 using label c for the vertex label.
                c        -- label of the edge
                a0       -- label of the vertex
    '''
    
    #fc = f(c,a0);
    fc = 1
    
    #build in the not's for the a0 check.
    binaried = binary_it(a0)
        
    #match binaried
    if len(binaried) > len(a):
        binaried = binaried[:len(a)]
    elif len(a) > len(binaried):
        for i in range(len(a)-len(binaried)):
            binaried.append(0)
    
    notMask = invert(binaried)
    
    for i in range(len(notMask)):
        if notMask[i]:
            circuit.x(a[i]);
    
    
    #build in the cnots for xoring b
    binaried = binary_it(fc)
        
    #match binaried
    if len(binaried) > len(r):
        binaried = binaried[:len(r)]
    elif len(r) > len(binaried):
        for i in range(len(r)-len(binaried)):
            binaried.append(0)
    
    for i in range(len(binaried)):
        if binaried[i]:
            circuit.mct(a,r[i], None, mode='noancilla')
        
    #build in the not's for the a0 return.
    for i in range(len(notMask)):
        if notMask[i]:
            circuit.x(a[i]);

# Black Box Algorithm

We demonstrate the idea by implementing two function calls in the image below.
Prior to the first barrier we implement, the output b xor 1, if and only if a = 1.

Past the barrier, we implement r xor 1 if and only if a = 2.

In [12]:
n = 2

simulator = Aer.get_backend('qasm_simulator')

a = QuantumRegister(n, 'a')
b = QuantumRegister(n, 'b')
r = QuantumRegister(1, 'r')

circuit = QuantumCircuit(a, b, r)

a0 = 1
xor_gate_v_each(circuit, a,b,r, 0,0,a0)
circuit.barrier()
a0 = 2
xor_gate_f_each(circuit, a,b,r, 0,0,a0)
circuit.draw()

# Implementing the Quantum Walk

The transformed quantum walk circuit requires only simple 2 qubit gates, CNOTs, and a controlled phase gate.

Combining these two features, in the future, we would like to implement the full quantum walk algorithm on arbitrary graphs.